In [4]:
import pandas as pd
import requests
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import logging
from unidecode import unidecode
import re

In [ ]:
# Configuración de logging
# logging.basicConfig(filename='geocoding_errors.log', level=logging.ERROR)

In [2]:
# Cargar el archivo CSV
df = pd.read_csv('datos_casinos_salas.csv')

# Mostrar algunas filas para entender el formato de las direcciones
print(df[['Dirección', 'Distrito', 'Provincia', 'Departamento']].head())

                           Dirección    Distrito Provincia Departamento
0          ALFREDO BENAVIDES No. 430  MIRAFLORES      LIMA         LIMA
1     Calle SAN MARTIN No. 275 - 275      TUMBES    TUMBES       TUMBES
2       Av. JOSE PARDO No. 422 - 422  MIRAFLORES      LIMA         LIMA
3  Av. MALECON DE LA RESERVA No. 615  MIRAFLORES      LIMA         LIMA
4      Av. LA MARINA No. 1725 - 1729  SAN MIGUEL      LIMA         LIMA


In [3]:
def clean_address(row):
    # Combinar campos relevantes
    address = row['Dirección']
    district = row['Distrito']
    province = row['Provincia']
    department = row['Departamento']

    # Limpieza básica
    address = re.sub(r'No\.?|N°|Nº|Nro\.?', '#', address)
    address = re.sub(r'\s+', ' ', address).strip()

    # Formato estándar: "Dirección, Distrito, Provincia, Departamento, Perú"
    full_address = f"{address}, {district}, {province}, {department}, Perú"

    return full_address

# Aplicar limpieza
df['Direccion_Limpia'] = df.apply(clean_address, axis=1)

In [7]:
print(df['Direccion_Limpia'])

0      ALFREDO BENAVIDES # 430, MIRAFLORES, LIMA, LIM...
1      Calle SAN MARTIN # 275 - 275, TUMBES, TUMBES, ...
2      Av. JOSE PARDO # 422 - 422, MIRAFLORES, LIMA, ...
3      Av. MALECON DE LA RESERVA # 615, MIRAFLORES, L...
4      Av. LA MARINA # 1725 - 1729, SAN MIGUEL, LIMA,...
                             ...                        
687    Av. LA MARINA # 662 y Calle SANTA CECILIA # 10...
688    Av. CARLOS IZAGUIRRE , LOTE 24 Mz. C, INDEPEND...
689    Av. PERSHING # 610 - 612, MAGDALENA DEL MAR, L...
690    Av. LA ENCALADA NROS 1415-1587 TIENDAS D-102, ...
691    Av. FRANCISCO BOLOGNESI # 756, CHICLAYO, CHICL...
Name: Direccion_Limpia, Length: 692, dtype: object


In [8]:
display(df)

,Ruc,Empresa,Establecimiento,Giro,Resolución,Código Sala,Vigencia,Dirección,Distrito,Provincia,Departamento,Direccion_Limpia,Latitud,Longitud
0,20265815830,CORPORACION TURISTICA PERUANA S.A.C.,ATLANTIC CITY,REST. 5TT,5363,110019002,20/01/2028,ALFREDO BENAVIDES No. 430,MIRAFLORES,LIMA,LIMA,"ALFREDO BENAVIDES # 430, MIRAFLORES, LIMA, LIM...",-12.124746,-77.028148
1,20231843460,COSTA DEL SOL S.A.,MASARIS - TUMBES,HOTEL ****,117,110021001,17/01/2027,Calle SAN MARTIN No. 275 - 275,TUMBES,TUMBES,TUMBES,"Calle SAN MARTIN # 275 - 275, TUMBES, TUMBES, ...",-3.508010,-80.270334
2,20305556786,GAMING AND SERVICES S.A.C.,JOKERS,HOTEL *****,1598,110032006,22/09/2026,Av. JOSE PARDO No. 422 - 422,MIRAFLORES,LIMA,LIMA,"Av. JOSE PARDO # 422 - 422, MIRAFLORES, LIMA, ...",-12.119149,-77.032302
3,20305556786,GAMING AND SERVICES S.A.C.,FANTASTIC,HOTEL *****,1614,110032007,25/10/2026,Av. MALECON DE LA RESERVA No. 615,MIRAFLORES,LIMA,LIMA,"Av. MALECON DE LA RESERVA # 615, MIRAFLORES, L...",-12.135230,-77.024922
4,20305556786,GAMING AND SERVICES S.A.C.,MIAMI,REST. 5TT,2917,110032008,11/07/2027,Av. LA MARINA No. 1725 - 1729,SAN MIGUEL,LIMA,LIMA,"Av. LA MARINA # 1725 - 1729, SAN MIGUEL, LIMA,...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,20545188202,ABSOLUTE GAMES S.A.C.,LUCKIA,REST. 5TT,819 - 2024,211544051,20/02/2028,Av. LA MARINA No. 662 y Calle SANTA CECILIA No...,PUEBLO LIBRE,LIMA,LIMA,Av. LA MARINA # 662 y Calle SANTA CECILIA # 10...,NaN,NaN
688,20549706500,DREAMS CORPORATION S.A.C.,PACHANGA,REST. 5TT,472,211668053,13/02/2027,"Av. CARLOS IZAGUIRRE , LOTE 24 Mz. C",INDEPENDENCIA,LIMA,LIMA,"Av. CARLOS IZAGUIRRE , LOTE 24 Mz. C, INDEPEND...",NaN,NaN
689,20549706500,DREAMS CORPORATION S.A.C.,NEW YORK CASINO,REST. 5TT,003775 - 2017,231668051,13/02/2027,Av. PERSHING No. 610 - 612,MAGDALENA DEL MAR,LIMA,LIMA,"Av. PERSHING # 610 - 612, MAGDALENA DEL MAR, L...",-12.087030,-77.062425
690,20611137355,CORPORACION EMPRESARIAL HOLDING S.A.C.,WIN & WIN,REST. 5TT,632 - 2025,232633001,31/01/2030,"Av. LA ENCALADA NROS 1415-1587 TIENDAS D-102, ...",SANTIAGO DE SURCO,LIMA,LIMA,"Av. LA ENCALADA NROS 1415-1587 TIENDAS D-102, ...",NaN,NaN


In [6]:
from geopy.geocoders import Nominatim
from time import sleep

# Configuración del geocodificador
geolocator = Nominatim(user_agent="mi_app_geocodificacion")

def get_coordinates(address):
    try:
        location = geolocator.geocode(address)
        if location:
            print(f"✅ Encontrado: {address}")
            return (location.latitude, location.longitude)
        else:
            print(f"❌ No encontrado: {address}")
            return (None, None)
    except Exception as e:
        print(f"⚠️ Error en {address}: {str(e)}")
        sleep(5)  # Espera antes de reintentar
        return (None, None)

# Aplicar geocodificación
df[['Latitud', 'Longitud']] = df['Direccion_Limpia'].apply(
    lambda x: pd.Series(get_coordinates(x))
)

✅ Encontrado: ALFREDO BENAVIDES # 430, MIRAFLORES, LIMA, LIMA, Perú
✅ Encontrado: Calle SAN MARTIN # 275 - 275, TUMBES, TUMBES, TUMBES, Perú
✅ Encontrado: Av. JOSE PARDO # 422 - 422, MIRAFLORES, LIMA, LIMA, Perú
✅ Encontrado: Av. MALECON DE LA RESERVA # 615, MIRAFLORES, LIMA, LIMA, Perú
⚠️ Error en Av. LA MARINA # 1725 - 1729, SAN MIGUEL, LIMA, LIMA, Perú: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Av.+LA+MARINA+%23+1725+-+1729%2C+SAN+MIGUEL%2C+LIMA%2C+LIMA%2C+Per%C3%BA&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
✅ Encontrado: Av. ARENALES 1814-A # 1814, LINCE, LIMA, LIMA, Perú
✅ Encontrado: Av. CAYMA # 701, CAYMA, AREQUIPA, AREQUIPA, Perú
✅ Encontrado: Calle MERCADERES # 303, AREQUIPA, AREQUIPA, AREQUIPA, Perú
❌ No encontrado: Otro PORTAL DE FLORES # 104 - 106 - 110 - 116 - 118 - 120, AREQUIPA, AREQUIPA, AREQUIPA, 

In [9]:
import numpy as np

# 1. Contar los resultados
# Contamos cuántos valores en la columna 'Latitud' no son nulos (encontrados)
encontrados = df['Latitud'].notna().sum()
# Contamos cuántos son nulos (no encontrados o con error)
no_encontrados = df['Latitud'].isna().sum()

print("--- Resumen de Geocodificación ---")
print(f"✅ Direcciones encontradas: {encontrados}")
print(f"❌ Direcciones no encontradas o con error: {no_encontrados}")
print("------------------------------------")

--- Resumen de Geocodificación ---
✅ Direcciones encontradas: 316
❌ Direcciones no encontradas o con error: 376
------------------------------------


In [10]:
# 2. Guardar los resultados en un archivo CSV
# Creamos una copia para no modificar el DataFrame original
df_resultados = df.copy()

# Creamos la columna 'Estado' basándonos en si se obtuvo o no una latitud
# np.where(condición, valor_si_verdadero, valor_si_falso)
df_resultados['Estado'] = np.where(df_resultados['Latitud'].notna(), 
                                'Encontrado', 
                                'No Encontrado / Error')

# Seleccionamos y reordenamos las columnas que queremos guardar
columnas_a_guardar = ['Direccion_Limpia', 'Estado', 'Latitud', 'Longitud']
df_para_csv = df_resultados[columnas_a_guardar]

# Definimos el nombre del archivo
nombre_archivo = 'resultados_casinos_geocodificados.csv'

# Guardamos el DataFrame en un archivo CSV
df_para_csv.to_csv(nombre_archivo, index=False, encoding='utf-8')

print(f"\n🎉 ¡Éxito! Los resultados han sido guardados en el archivo: '{nombre_archivo}'")

# Mostramos las primeras 5 filas del archivo que se guardó
print("\nVista previa de los datos guardados:")
print(df_para_csv.head())


🎉 ¡Éxito! Los resultados han sido guardados en el archivo: 'resultados_casinos_geocodificados.csv'

Vista previa de los datos guardados:
                                    Direccion_Limpia                 Estado  \
0  ALFREDO BENAVIDES # 430, MIRAFLORES, LIMA, LIM...             Encontrado   
1  Calle SAN MARTIN # 275 - 275, TUMBES, TUMBES, ...             Encontrado   
2  Av. JOSE PARDO # 422 - 422, MIRAFLORES, LIMA, ...             Encontrado   
3  Av. MALECON DE LA RESERVA # 615, MIRAFLORES, L...             Encontrado   
4  Av. LA MARINA # 1725 - 1729, SAN MIGUEL, LIMA,...  No Encontrado / Error   

     Latitud   Longitud  
0 -12.124746 -77.028148  
1  -3.508010 -80.270334  
2 -12.119149 -77.032302  
3 -12.135230 -77.024922  
4        NaN        NaN  
